In [ ]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [ ]:
years = []
holidays = {}

country_id = "UK"
country = "ENG"
wc = "MON"
start_date = datetime.strptime(r"01/01/2018", r"%d/%m/%Y").strftime(r"%d/%m/%Y")
end_date = datetime.strptime(r"31/08/2023", r"%d/%m/%Y").strftime(r"%d/%m/%Y")

start_year = int(datetime.strptime(start_date, r"%d/%m/%Y").strftime(r"%Y"))
end_year = int(datetime.strptime(end_date, r"%d/%m/%Y").strftime(r"%Y"))

while start_year <= end_year:
    years.append(start_year)
    start_year += 1

## Helper functions

In [ ]:
def date_convertion(wc, date):
    if wc == "MON":
        if date.weekday() != 0:
            date = date - timedelta(date.weekday())
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")
    elif wc == "TUE":
        if date.weekday() != 1:
            date = date - timedelta(date.weekday() - 1)
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")
    elif wc == "WED":
        if date.weekday() != 2:
            date = date - timedelta(date.weekday() - 2)
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")
    elif wc == "THU":
        if date.weekday() != 3:
            date = date - timedelta(date.weekday() - 3)
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")
    elif wc == "FRI":
        if date.weekday() != 4:
            date = date - timedelta(date.weekday() - 4)
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")
    elif wc == "SAT":
        if date.weekday() != 5:
            date = date - timedelta(date.weekday() - 5)
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")
    elif wc == "SUN":
        if date.weekday() != 6:
            date = date - timedelta(date.weekday() - 6)
            return date.strftime(r"%d/%m/%Y")
        else:
            return date.strftime(r"%d/%m/%Y")

## Weekly

### Creates the DataFrame

In [ ]:
dates = pd.date_range(start= start_date, end= end_date, freq= "D")
df = pd.DataFrame({"date": dates})

df.head()

### Adds the 'Week #' column

In [ ]:
week_numbers = []

for (index, row) in df.iterrows():
    week_numbers.append(row["date"].isocalendar().week)
df["Week #"] = week_numbers

df.head()

### Converts all dates into W/C monday

In [ ]:
logic = {"Week #": "min"}
df = df.resample("W-MON", label= "left", closed= "left", on= "date").apply(logic)
df.head()

### Creates the weekly dummies and resets the index

In [ ]:
weeks = {}
for i in range(1, 54):
    week = f"Week {i}"
    if week not in weeks: 
        weeks[week] = []

    for (index, row) in df.iterrows():
        if row["Week #"] == i:
            weeks[week].append(1)
        else:
            weeks[week].append(0)

    df[f"Dummy {week}"] = weeks[week]

df.reset_index(inplace= True)
df.head()

### Changes the dates formats and creates the CSV file

In [ ]:
df["date"] = df["date"].dt.strftime(r"%d/%m/%Y")
df.head()

In [ ]:
df.to_csv(r"C:\Users\nicolas.kossacoff\Documents\Python\Seasonality\Weekly Dummies (testing).csv", index= False)

## Month

In [ ]:
dates = pd.date_range(start= start_date, end= end_date, freq= "D")
df1 = pd.DataFrame({"Date": dates})

month_numbers = []
for (index, row) in df1.iterrows():
    month_numbers.append(row["Date"].month)

df1["Month #"] = month_numbers
df1.head()

In [ ]:
months = {}
months_name = ["January", "February", "March", "April", "May", "June", 
          "July", "August", "September", "October", "November", "December"]

for i in range(1,13):
    month = months_name[i - 1]

    if month not in months: 
        months[month] = []

    for (index, row) in df1.iterrows():
        if row["Month #"] == i:
            months[month].append(1)
        else:
            months[month].append(0)

    df1[f"Dummy {month}"] = months[month]

df1.head()

In [ ]:
df1 = df1.drop(columns= ["Month #"])
df1.head()

In [ ]:
#logic = {"Dummy January": "min"}
df1 = df1.resample(f"W-MON", label= "left", closed= "left", on= "Date").sum()
df1.head()

In [ ]:
for column in df1.columns:
    df1[column] = (df1[column] > 3).astype(int)

df1.head()

In [ ]:
df1.reset_index(inplace= True)
df1["Date"] = df1["Date"].dt.strftime(r"%d/%m/%Y")
df1.head()

In [ ]:
df1.rename(columns= {"Date": "date"}, inplace= True)
df1.head()

In [ ]:
df1.to_csv(r"C:\Users\nicolas.kossacoff\Documents\Python\Seasonality\Workings and Testings\CSVs\Monthly Dummies (testings).csv", index= False)

## Joining DataFrame

In [ ]:
df = df.merge(df1, how= "inner", on= "date")
df.head()

In [ ]:
df.to_csv(r"C:\Users\nicolas.kossacoff\Documents\Python\Seasonality\Workings and Testings\CSVs\Joined DataFrames (testing).csv", index= False)